In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 19220866
paper_name = 'mira_sa_correia_2009' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
files = ['hits_genenames.txt','hits_genenames_moderate.txt']

In [6]:
original_data_list = []
for f in files:
    original_data = pd.read_csv('raw_data/' + f, header=None, sep='\t')
    print('Original data dimensions: %d x %d' % (original_data.shape))
#     print(original_data.head())
    original_data['gene'] = original_data[0].astype(str)
    original_data['gene'] = clean_genename(original_data['gene'])
    original_data['orf'] = translate_sc(original_data['gene'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    original_data = original_data.loc[t,:]
    original_data.set_index('orf', inplace=True)
    original_data['data'] = -1
    original_data = original_data[['data']].copy()
    original_data_list.append(original_data)

Original data dimensions: 270 x 1
                 0   gene    orf
index_input                     
111          MRP16  MRP16  MRP16
Original data dimensions: 43 x 1
Empty DataFrame
Columns: [0, gene, orf]
Index: []


In [7]:
original_data = pd.concat(original_data_list, axis=1)

In [8]:
original_data.columns=['all','moderate']
original_data[original_data.isnull()] = 0
original_data['data'] = original_data['all'] - original_data['moderate'] - 1

In [9]:
original_data.index.name='orf'
original_data = original_data[['data']]

In [10]:
original_data = original_data.groupby(original_data.index).mean()

In [11]:
original_data.shape

(269, 1)

# Load & process tested strains

In [12]:
# tested = pd.read_excel('raw_data/List of strains tested.xlsx', sheet_name='Tabelle2')

In [13]:
# tested.head()

In [14]:
# tested['orf'] = tested['ORF'].astype(str)

In [15]:
# tested['orf'] = clean_orf(tested['orf'])

In [16]:
# tested['orf'] = translate_sc(tested['orf'], to='orf')

In [17]:
# # Make sure everything translated ok
# t = looks_like_orf(tested['orf'])
# print(tested.loc[~t,])

In [18]:
# tested_orfs = tested['orf'].unique()

In [19]:
# missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
# missing

In [20]:
# tested_orfs = list(tested_orfs) + missing

In [21]:
# original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [22]:
data = original_data.copy()

In [23]:
dataset_ids = [157]
datasets = datasets.reindex(index=dataset_ids)

In [24]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [25]:
data.head()

dataset_id,157
data_type,value
orf,
YAL009W,-2.0
YAL048C,-2.0
YAL059W,-2.0
YAR015W,-2.0
YBL012C,-2.0


## Subset to the genes currently in SGD

In [26]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 2


In [27]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,157
,data_type,value
gene_id,orf,
7,YAL009W,-2.0
46,YAL048C,-2.0
54,YAL059W,-2.0
67,YAR015W,-2.0
100,YBL012C,-2.0


# Normalize

In [28]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [29]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [30]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        157          
data_type       value    valuez
gene_id orf                    
7       YAL009W  -2.0 -4.411236
46      YAL048C  -2.0 -4.411236
54      YAL059W  -2.0 -4.411236
67      YAR015W  -2.0 -4.411236
100     YBL012C  -2.0 -4.411236

# Print out

In [31]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [35]:
# from IO.save_data_to_db3 import *

In [36]:
# save_data_to_db(data_all, paper_pmid)